In [1]:
'''
! pip install torchmetrics
! pip install watermark
! pip install mlxtend
! pip install pandas'''

'! pip install lightning\n! pip install torchmetrics\n! pip install watermark\n! pip install mlxtend\n! pip install tensorboard\n! pip install pandas'

In [7]:
import torch

from config import config
from train import train
from models import RACNet
from plotting import show_failures, plot_loss_and_acc

/home/karanjotvendal/miniforge3/envs/thesis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%reload_ext watermark
%watermark -a 'Karanjot Vendal' -v -p torch --iversion

Author: Karanjot Vendal

Python implementation: CPython
Python version       : 3.11.4
IPython version      : 8.14.0

torch: 2.0.1

torch: 2.0.1



In [3]:
import wandb
wandb.login(key=config.wandb_key)

wandb: Currently logged in as: karanjot. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/karanjotvendal/.netrc


True

# Hyperparameters

In [4]:
PATH = 'data/reduced_dataset/'
MODEL = 'resnet18'
BATCH_SIZE = 3
NUM_EPOCHS = 5
LEARNING_RATE = 0.0001

NUM_WORKERS = 2
NUM_CLASSES = 2

KFOLD= 10 
# Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
run = wandb.init(

      project="RACNet", 
      name=f"experiment_{'sanity_check'}",
      notes="Changed the way metrics are computed for test set.",
      config={
      "learning_rate": LEARNING_RATE,
      "architecture": "CNN-GRU-MASK-FC-Classifier",
      "dataset": "MICAA MRI",
      "epochs": NUM_EPOCHS,
      "BATCH_SIZE": BATCH_SIZE
      })

# Model

In [6]:
model = RACNet(MODEL, NUM_CLASSES)
model

RACNet(
  (cnn): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bi

# Training

In [ ]:
train(MODEL, LEARNING_RATE, NUM_CLASSES, PATH, NUM_EPOCHS, KFOLD, BATCH_SIZE, NUM_WORKERS, DEVICE)

--------------FOLD:1-----------------------
--------------------------------------------------
[tensor([35, 18, 33])]
feature shape torch.Size([3, 254, 1, 112, 112])
reshape input torch.Size([762, 1, 112, 112])
CNN ouput torch.Size([762, 512])
reshaped rnn_in torch.Size([3, 254, 512])
RNN ouput torch.Size([3, 254, 64])
mask ouput torch.Size([3, 254, 64])
reshaped masked output torch.Size([3, 16256])
fc ouput torch.Size([3, 32])
logits torch.Size([3, 2])
classifier ouput torch.Size([3, 2])


# Evaluating the model

In [ ]:
#visualising cofusion matrix
from torchmetrics import ConfusionMatrix
import matplotlib
from mlxtend.plotting import plot_confusion_matrix

cmat = ConfusionMatrix(task='multiclass', num_classes=NUM_CLASSES)

for x, y in dm.test_dataloader():

    with torch.no_grad():
        pred = lightning_model(x)
    cmat(pred, y)

cmat_tensor = cmat.compute()
cmat = cmat_tensor.numpy()

fig, ax = plot_confusion_matrix(
    conf_mat=cmat,
    class_names=class_dict.values(),
    norm_colormap=matplotlib.colors.LogNorm()  
    # normed colormaps highlight the off-diagonals 
    # for high-accuracy models better
)

plt.show()

In [26]:
import cv2

In [27]:
def load_image(path, size=(112,112)):
    image = cv2.imread(path, 0)
    if image is None:
        return np.zeros()
    
    image = cv2.resize(image, size) / 255
    return image.astype('f')

In [28]:
paths =[]
images=[load_image(path) for path in paths]
org_size = len(images)

In [31]:
dup_len = 254 - len(images)
if org_size == 0:
    dup = torch.zeros(1, 112, 112)
    print('dup', dup)
else:
    dup = images[-1]
for i in range(dup_len):
    images.append(dup)

images = [torch.tensor(image, dtype=torch.float32) for image in images]
images = torch.stack(images)



dup tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


/tmp/ipykernel_3239/2694937284.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image, dtype=torch.float32) for image in images]


In [32]:
images.shape

torch.Size([254, 1, 112, 112])

In [16]:
img = load_image('data/reduced_dataset/00002/FLAIR/Image-437.png')

In [17]:
img.shape

(112, 112)

In [19]:
from PIL import Image

In [23]:
ig = plt.imread('data/reduced_dataset/00002/FLAIR/Image-437.png')

In [25]:
ig

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [18]:
image = []
image.append(img)
image

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)]

In [19]:
dup = image[-1]
dup


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [20]:
for i in range(4):
    image.append(dup)

image

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],


In [25]:
image = [torch.tensor(image, dtype=torch.float32) for image in image]


In [26]:
image

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 

In [27]:
image[0]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [29]:
image[0].shape


torch.Size([112, 112])

In [30]:
image = torch.stack(image)

In [31]:
image.shape

torch.Size([5, 112, 112])

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [35]:
images = []
img = load_image('data/reduced_dataset/00002/FLAIR/Image-437.png')
images.append(img)

dup = images[-1]

for i in range(4):
    images.append(dup)

images = [torch.tensor(image, dtype=torch.float32) for image in images]
images = torch.vstack(images)

In [37]:
images

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [4]:
org = [0]
masks= []
for i in org:
    #print(i)
    dup = 254 - i
    mask_1 = torch.ones(i, 64) # .to(device='cuda')
    mask_0 = torch.zeros(dup, 64) #.to(device='cuda')
    mask = torch.cat((mask_1, mask_0), 0)
    masks.append(mask)

masks = torch.stack(masks)
mask.shape

torch.Size([254, 64])

In [3]:
n = np.zeros((1,112,112))

In [2]:
images = []
org_size = 150
dup_len = 254 - org_size
dup = np.zeros((1,112,112))
for i in range(150):
    images.append(dup)

dup = images[-1]
for i in range(dup_len):
    images.append(dup)

images = [torch.tensor(image, dtype=torch.float32) for image in images]
images = torch.stack(images)

In [3]:
images.shape

torch.Size([254, 1, 112, 112])

In [4]:
images = []
dup = np.zeros((1,112,112))  #torch.zeros(1, 112, 112).numpy()

for i in range(254):
    images.append(dup)

In [11]:
len(images)

#[tensor([0])]
#feature shape torch.Size([1, 254, 1, 1, 112, 112])

254

In [5]:
images = [torch.tensor(image, dtype=torch.float32) for image in images]
images = torch.stack(images)
images.shape

torch.Size([254, 1, 112, 112])

In [4]:
data =[]
data.append(images)
data = torch.stack(data)
data.shape

torch.Size([1, 254, 1, 112, 112])

In [9]:
data.transpose(1,0).shape

torch.Size([254, 1, 1, 112, 112])

In [10]:
d =[]
d.append(images)
d = torch.stack(d)
d.shape

torch.Size([1, 254, 1, 112, 112])

In [11]:
d.transpose(1,0).shape

torch.Size([254, 1, 1, 112, 112])

In [11]:
from collections import Counter
import numpy as np
from datamodules import RSNAdataset

from torch.utils.data import DataLoader, Dataset

In [12]:
for _ in range(10):
    
    fold = _+1
    print(f'--------------FOLD:{fold}-----------------------') 

    folds_xtrain = np.load('./data/folds/new_folds/xtrain.npy', allow_pickle=True)
    folds_xtest = np.load('./data/folds/new_folds/xtest.npy', allow_pickle=True)
    folds_ytrain = np.load('./data/folds/new_folds/ytrain.npy', allow_pickle=True)
    folds_ytest = np.load('./data/folds/new_folds/ytest.npy', allow_pickle=True)
        
    xtrain = folds_xtrain[_]
    ytrain = folds_ytrain[_]
    xtest = folds_xtest[_]
    ytest = folds_ytest[_]

    train_set = RSNAdataset(
                    './data/reduced_dataset/',
                    xtrain,  
                    ytrain,
                    n_slices=254,
                    img_size=112,
                    transform=None
                        )

    test_set = RSNAdataset(
                    './data/reduced_dataset/',
                    xtest,  
                    ytest,
                    n_slices=254,
                    img_size=112,
                    transform=None
                        )
    
    train_loader = DataLoader(
                train_set,    
                batch_size=5,
                shuffle=True,
                num_workers=0,
            )
    
    test_loader = DataLoader(
                test_set,    
                batch_size=5,
                shuffle=False,
                num_workers=0,
            )
        
    train_counter = Counter()
    for idx, batch in enumerate(train_loader):
        train_counter.update(batch['y'].tolist())
        
    print("\nTraining label distribution:")
    print(sorted(train_counter.items()))    
    
    test_counter = Counter()
    for idx, batch in enumerate(test_loader):
        test_counter.update(batch['y'].tolist())
    
    print("\nTest label distribution:")
    print(sorted(test_counter.items()))

--------------FOLD:1-----------------------

Training label distribution:
[(0, 221), (1, 244)]

Test label distribution:
[(0, 55), (1, 62)]
--------------FOLD:2-----------------------

Training label distribution:
[(0, 221), (1, 244)]

Test label distribution:
[(0, 55), (1, 62)]
--------------FOLD:3-----------------------

Training label distribution:
[(0, 221), (1, 244)]

Test label distribution:
[(0, 55), (1, 62)]
--------------FOLD:4-----------------------

Training label distribution:
[(0, 221), (1, 244)]

Test label distribution:
[(0, 55), (1, 62)]
--------------FOLD:5-----------------------

Training label distribution:
[(0, 221), (1, 244)]

Test label distribution:
[(0, 55), (1, 62)]
--------------FOLD:6-----------------------

Training label distribution:
[(0, 221), (1, 244)]

Test label distribution:
[(0, 55), (1, 62)]
--------------FOLD:7-----------------------

Training label distribution:
[(0, 221), (1, 244)]

Test label distribution:
[(0, 55), (1, 62)]
--------------FOLD:8